In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# 'c:\\LearnMachineLearning\\Kaggle\\Titanic'
import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./Data Hunt.ipynb
./debug.log
./Example Run.ipynb
./Expanded Data Attempt.ipynb
./gender_submission.csv
./test.csv
./Titanic Example.ipynb
./train.csv
./.ipynb_checkpoints\Data Hunt-checkpoint.ipynb
./.ipynb_checkpoints\Example Run-checkpoint.ipynb
./.ipynb_checkpoints\Expanded Data Attempt-checkpoint.ipynb
./.ipynb_checkpoints\Titanic Example-checkpoint.ipynb
./ExampleSubmission\my_submission.csv
./MySubmissions\HasAge-Deck-TickGroup.csv
./MySubmissions\splitAgeToBoolean.csv


In [2]:
train_data = pd.read_csv("./train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("./test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# print(pd.options.display.max_rows) # 10
# print(pd.options.display.max_columns) # 20
# print(pd.options.display.width) # 80
# print(pd.options.display.max_colwidth) # 50

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

types = train_data.dtypes
print(types)

count_nan = len(train_data) - train_data.count()
print(count_nan)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [5]:
import operator

print(operator.eq)

def value_rate(column, op, value, string_name):
    value = train_data.loc[op(train_data[column], value)]["Survived"]
    rate_value = sum(value)/len(value)
    print("% of", string_name, "who survived:", rate_value)

<built-in function eq>


In [6]:
# women = train_data.loc[train_data.Sex == 'female']["Survived"]
# rate_women = sum(women)/len(women)
# print("% of women who survived:", rate_women)
value_rate('Sex', operator.eq, 'female', 'women')

value_rate('Sex', operator.eq, 'male', 'men')

value_rate('Pclass', operator.eq, 1, 'P 1')

value_rate('Pclass', operator.eq, 2, 'P 2')

value_rate('Pclass', operator.eq, 3, 'P 3')

value_rate('Age', operator.ge, 18, 'Adult')

value_rate('Age', operator.le, 7, 'child')

% of women who survived: 0.7420382165605095
% of men who survived: 0.18890814558058924
% of P 1 who survived: 0.6296296296296297
% of P 2 who survived: 0.47282608695652173
% of P 3 who survived: 0.24236252545824846
% of Adult who survived: 0.3810316139767055
% of child who survived: 0.68


In [7]:
# train_data_has_age = train_data[train_data['Age'].notna()]
tdha = train_data[train_data['Age'].notna()]
adult = train_data.loc[train_data['Age'].isna()]["Survived"] # & (tdha.Sex == 'male')
rate_adult = sum(adult)/len(adult)
print("% who survived:", rate_adult)

% who survived: 0.2937853107344633


In [8]:
p_classes = train_data.groupby(['Pclass']).size()
p_classes

Pclass
1    216
2    184
3    491
dtype: int64

In [9]:
cabins = train_data.groupby(['Cabin']).size()
cabins

Cabin
A10    1
A14    1
A16    1
A19    1
A20    1
      ..
F33    3
F38    1
F4     2
G6     4
T      1
Length: 147, dtype: int64

In [10]:
hasCabins = train_data[train_data['Cabin'].notna()]
decks = hasCabins["Cabin"].str.split(r"(\d+)", expand=True)[0]
decks

1      C
3      C
6      E
10     G
11     C
      ..
871    D
872    B
879    C
887    B
889    C
Name: 0, Length: 204, dtype: object

In [11]:
expanded_train_data = train_data
expanded_train_data = expanded_train_data.assign(has_age=expanded_train_data.Age.notna(),
                                                is_child=(expanded_train_data.Age.notna()) & (expanded_train_data['Age'] < 18),
                                                is_elderly=(expanded_train_data.Age.notna()) & (expanded_train_data.Age > 65),
                                                deck=expanded_train_data["Cabin"].str.split(r"(\d+| )",expand=True)[0],
                                                ticket_group = expanded_train_data.Fare // 10 * 10)
expanded_train_data # .groupby(['deck']).size()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,has_age,is_child,is_elderly,deck,ticket_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True,False,False,NaN,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True,False,False,C,70.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,True,False,False,NaN,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,True,False,False,C,50.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True,False,False,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,True,False,False,NaN,10.0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,True,False,False,B,30.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,False,False,False,NaN,20.0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,True,False,False,C,30.0


In [12]:
# train_data_has_age = train_data[train_data['Age'].notna()]
values = expanded_train_data.loc[expanded_train_data['has_age']]["Survived"]
rate_values = sum(values)/len(values)
print("% who survived:", rate_values)

% who survived: 0.4061624649859944


In [13]:
expanded_train_data[expanded_train_data.Fare < 10]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,has_age,is_child,is_elderly,deck,ticket_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True,False,False,NaN,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,True,False,False,NaN,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True,False,False,NaN,0.0
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,False,False,False,NaN,0.0
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S,True,False,False,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,878,0,3,"Petroff, Mr. Nedelio",male,19.0,0,0,349212,7.8958,NaN,S,True,False,False,NaN,0.0
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S,False,False,False,NaN,0.0
881,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S,True,False,False,NaN,0.0
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S,True,False,False,NaN,0.0


In [14]:
expanded_train_data.groupby(expanded_train_data.Fare // 10 * 10).size()

Fare
0.0      336
10.0     179
20.0     136
30.0      64
40.0      15
50.0      39
60.0      17
70.0      29
80.0      15
90.0       8
100.0      4
110.0      7
120.0      4
130.0      7
140.0      2
150.0      7
160.0      2
210.0      4
220.0      5
240.0      2
260.0      6
510.0      3
dtype: int64

In [15]:
value = expanded_train_data.loc[expanded_train_data.Fare // 10 * 10 == 510.0]["Survived"]
rate_value = sum(value)/len(value)
print("% who survived:", rate_value)

% who survived: 1.0


In [17]:
expanded_train_data = train_data
expanded_train_data = expanded_train_data.assign(has_age=expanded_train_data.Age.notna(),
                                                is_child=(expanded_train_data.Age.notna()) & (expanded_train_data['Age'] < 18),
                                                is_elderly=(expanded_train_data.Age.notna()) & (expanded_train_data.Age > 65),
                                                deck= expanded_train_data["Cabin"].str.split(r"(\d+| )",expand=True)[0].fillna(' ').apply(lambda x: ord(x.lower())-98),
                                                ticket_group = (expanded_train_data.Fare // 10 * 10).fillna(0.0))


expanded_train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,has_age,is_child,is_elderly,deck,ticket_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True,False,False,-66,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True,False,False,1,70.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,True,False,False,-66,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,True,False,False,1,50.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True,False,False,-66,0.0


In [18]:
expanded_test_data = test_data
expanded_test_data = expanded_test_data.assign(has_age=expanded_test_data.Age.notna(),
                                                is_child=(expanded_test_data.Age.notna()) & (expanded_test_data['Age'] < 18),
                                                is_elderly=(expanded_test_data.Age.notna()) & (expanded_test_data.Age > 65),
                                                deck=expanded_test_data["Cabin"].str.split(r"(\d+| )",expand=True)[0].fillna(' ').apply(lambda x: ord(x.lower())-98),
                                                ticket_group = (expanded_test_data.Fare // 10 * 10).fillna(0.0))
expanded_test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,has_age,is_child,is_elderly,deck,ticket_group
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,True,False,False,-66,0.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,True,False,False,-66,0.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,True,False,False,-66,0.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,True,False,False,-66,0.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,True,False,False,-66,10.0


In [25]:
deck=expanded_test_data["Cabin"].str.split(r"(\d+| )",expand=True)[0].fillna(' ').apply(lambda x: print(x.lower(), ord(x.lower())-97))
deck

  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
b 1
  -65
e 4
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
b 1
  -65
b 1
  -65
a 0
  -65
  -65
  -65
  -65
  -65
c 2
  -65
  -65
  -65
  -65
  -65
  -65
d 3
  -65
  -65
d 3
  -65
a 0
  -65
d 3
  -65
c 2
  -65
  -65
c 2
  -65
  -65
  -65
f 5
  -65
b 1
  -65
  -65
  -65
  -65
b 1
  -65
  -65
  -65
c 2
c 2
  -65
  -65
  -65
d 3
c 2
c 2
  -65
c 2
  -65
  -65
  -65
c 2
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
b 1
  -65
  -65
  -65
c 2
  -65
  -65
  -65
c 2
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
f 5
  -65
  -65
a 0
  -65
c 2
  -65
  -65
g 6
c 2
  -65
  -65
  -65
c 2
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
c 2
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
b 1
  -65
  -65
  -65
e 4
  -65
  -65
  -65
c 2
  -65
  -65
  -65
  -65
  -65
c 2
  -65
d 3
  -65
  -65
  -65
  -65
  -65
  -65
  -65
b 1
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
  -65
c 2
f 5
e 4
  

0      None
1      None
2      None
3      None
4      None
       ... 
413    None
414    None
415    None
416    None
417    None
Name: 0, Length: 418, dtype: object